In [1]:
import xgboost
import lightgbm
import torch
import numpy as np
import pandas as pd
import datetime
import torch.nn.functional as F
from sklearn.model_selection import StratifiedKFold

In [2]:
IMAGE_TRAIN_PRE='../data/AgriculturalDisease_trainingset/images/'
ANNOTATION_TRAIN='../data/AgriculturalDisease_trainingset/AgriculturalDisease_train_annotations_deleteNoise.json' #是否需要剔除两类异常类
IMAGE_VAL_PRE='../data/AgriculturalDisease_validationset/images/'
ANNOTATION_VAL='../data/AgriculturalDisease_validationset/AgriculturalDisease_validation_annotations_deleteNoise.json' #是否需要剔除两类异常类
date=str(datetime.date.today())
with open(ANNOTATION_TRAIN) as datafile1:
    trainDataFram=pd.read_json(datafile1,orient='records')
with open(ANNOTATION_VAL) as datafile2: #first check if it's a valid json file or not
    validateDataFram =pd.read_json(datafile2,orient='records')

In [3]:
# labels=validateDataFram['disease_class'].values
# predict=torch.load('../feature/Resnet50/val_raw_prediction.pth')
# predictlabel=predict['lx'].numpy()
# # (predictlabel==labels).all()

# predictScore=predict['px']
# print(predictScore)
# predictScoreSoftMax=F.softmax(predictScore,dim=1).numpy()
# print(predictScoreSoftMax.shape)
# predict2=torch.load('../feature/Resnet152/val_raw_prediction.pth')
# predictScore2=predict2['px']
# predictScoreSoftMax2=F.softmax(predictScore,dim=1).numpy()
# a=np.concatenate((predictScoreSoftMax,predictScoreSoftMax2),axis=1)
# np.concatenate((a,predictScoreSoftMax2),axis=1).shape

In [4]:
def load_data(featurePaths,dataFrame):
    label=dataFrame['disease_class'].values # array[1,1,1.....38]
    predictScoreBase=torch.load(featurePaths[0])['px'].numpy()
    if len(predictScoreBase.shape)==3:
        predictScoreBase=np.reshape(predictScoreBase,(predictScoreBase.shape[0],-1),order='F')
    for i in range(1,len(featurePaths)):
        predict=torch.load(featurePaths[i])
        predictScore=predict['px'].numpy()
        if len(predictScore)==3:
            predictScore=np.reshape(predictScore,(predictScore.shape[0],-1),order='F')
        predictScoreBase=np.concatenate((predictScoreBase,predictScore),1)
        print('size of predict is ',predictScoreBase.shape)
    return predictScoreBase,label
    

In [13]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split



def ligGBMWithfeatures(featurenPaths,dataFrame):
    


    train_val_x,train_val_y= load_data(featurenPaths,dataFrame)
    split_fold = StratifiedKFold(n_splits=4)
    folds_indexes=split_fold.split(train_val_x,train_val_y)
#     for i in folds_indexes:
    folds_indexes= np.array(list(folds_indexes))
    i=folds_indexes[0]
    X_train=train_val_x[i[0],:]
    y_train=train_val_y[i[0]]
    X_test = train_val_x[i[1],:]
    y_test=train_val_y[i[1]]

    # 用sklearn.cross_validation进行训练数据集划分，这里训练集和交叉验证集比例为7：3，可以自己根据需要设置
#     X, val_X, y, val_y = train_test_split(
#         train_x,
#         train_y,
#         test_size=0.05,
#         random_state=1,
#         stratify=train_y ## 这里保证分割后y的比例分布与原数据一致
#     )



    # create dataset for lightgbm
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    # specify your configurations as a dict
    params = {
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'num_class': 59,
        'max_bin': 255,
        'metric': 'multi_error',
        'num_leaves': 30,
        'learning_rate': 0.01,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'lambda_l1': 0.4,
        'lambda_l2': 0.5,
        'min_gain_to_split': 0.2,
        'verbose': 5,
        'is_unbalance': True
    }

    # train
    print('Start training...')
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=10000,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=500)
    gbm.save_model('model.txt')
    print('Start predicting...')
    
    preds = gbm.predict(X_test, num_iteration=gbm.best_iteration)  # 输出的是概率结果
    print('max acc is ',np.mean(np.argmax(preds,1)==y_test))
    return preds


In [10]:
# a=np.array([[[1,2,3],[4,5,6],[7,8,9]],[[1,2,3],[4,5,6],[10,11,12]]])
# np.reshape(a,(a.shape[0],-1),order='F')

In [15]:
featurePaths=['../feature/DesNet161/val_all_prediction.pth','../feature/DesNet201/val_raw_prediction.pth','../feature/Resnet152/val_raw_prediction.pth','../feature/Resnet50/val_raw_prediction.pth','../feature/InceptionResnet/val_raw_prediction.pth','../feature/Nasnetmobile/val_raw_prediction.pth','../feature/InceptionV3/val_raw_prediction.pth','../feature/Xception/val_raw_prediction.pth','../feature/InceptionV4/val_raw_prediction.pth']
ligGBMWithfeatures(featurePaths,validateDataFram)

size of predict is  (4539, 767)
size of predict is  (4539, 826)
size of predict is  (4539, 885)
size of predict is  (4539, 944)
size of predict is  (4539, 1003)
size of predict is  (4539, 1062)
size of predict is  (4539, 1121)
size of predict is  (4539, 1180)
Start training...
[1]	valid_0's multi_error: 0.202936
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's multi_error: 0.182211
[3]	valid_0's multi_error: 0.170984
[4]	valid_0's multi_error: 0.170984
[5]	valid_0's multi_error: 0.174439
[6]	valid_0's multi_error: 0.175302
[7]	valid_0's multi_error: 0.175302
[8]	valid_0's multi_error: 0.175302
[9]	valid_0's multi_error: 0.177029
[10]	valid_0's multi_error: 0.173575
[11]	valid_0's multi_error: 0.177029
[12]	valid_0's multi_error: 0.176166
[13]	valid_0's multi_error: 0.177029
[14]	valid_0's multi_error: 0.175302
[15]	valid_0's multi_error: 0.170984
[16]	valid_0's multi_error: 0.173575
[17]	valid_0's multi_error: 0.170984
[18]	valid_0's multi_error: 0.16753
[19

[212]	valid_0's multi_error: 0.15285
[213]	valid_0's multi_error: 0.15285
[214]	valid_0's multi_error: 0.15285
[215]	valid_0's multi_error: 0.15285
[216]	valid_0's multi_error: 0.151986
[217]	valid_0's multi_error: 0.151986
[218]	valid_0's multi_error: 0.151123
[219]	valid_0's multi_error: 0.151123
[220]	valid_0's multi_error: 0.150259
[221]	valid_0's multi_error: 0.150259
[222]	valid_0's multi_error: 0.150259
[223]	valid_0's multi_error: 0.151123
[224]	valid_0's multi_error: 0.151123
[225]	valid_0's multi_error: 0.150259
[226]	valid_0's multi_error: 0.149396
[227]	valid_0's multi_error: 0.149396
[228]	valid_0's multi_error: 0.149396
[229]	valid_0's multi_error: 0.149396
[230]	valid_0's multi_error: 0.148532
[231]	valid_0's multi_error: 0.148532
[232]	valid_0's multi_error: 0.149396
[233]	valid_0's multi_error: 0.150259
[234]	valid_0's multi_error: 0.150259
[235]	valid_0's multi_error: 0.150259
[236]	valid_0's multi_error: 0.150259
[237]	valid_0's multi_error: 0.150259
[238]	valid_0's 

[428]	valid_0's multi_error: 0.150259
[429]	valid_0's multi_error: 0.150259
[430]	valid_0's multi_error: 0.151986
[431]	valid_0's multi_error: 0.151986
[432]	valid_0's multi_error: 0.151986
[433]	valid_0's multi_error: 0.151986
[434]	valid_0's multi_error: 0.151986
[435]	valid_0's multi_error: 0.151986
[436]	valid_0's multi_error: 0.151986
[437]	valid_0's multi_error: 0.151986
[438]	valid_0's multi_error: 0.151123
[439]	valid_0's multi_error: 0.150259
[440]	valid_0's multi_error: 0.150259
[441]	valid_0's multi_error: 0.150259
[442]	valid_0's multi_error: 0.150259
[443]	valid_0's multi_error: 0.150259
[444]	valid_0's multi_error: 0.151123
[445]	valid_0's multi_error: 0.150259
[446]	valid_0's multi_error: 0.151123
[447]	valid_0's multi_error: 0.151123
[448]	valid_0's multi_error: 0.150259
[449]	valid_0's multi_error: 0.150259
[450]	valid_0's multi_error: 0.151123
[451]	valid_0's multi_error: 0.151123
[452]	valid_0's multi_error: 0.151123
[453]	valid_0's multi_error: 0.151123
[454]	valid_

[646]	valid_0's multi_error: 0.154577
[647]	valid_0's multi_error: 0.154577
[648]	valid_0's multi_error: 0.154577
[649]	valid_0's multi_error: 0.154577
[650]	valid_0's multi_error: 0.154577
[651]	valid_0's multi_error: 0.154577
[652]	valid_0's multi_error: 0.154577
[653]	valid_0's multi_error: 0.15544
[654]	valid_0's multi_error: 0.156304
[655]	valid_0's multi_error: 0.156304
[656]	valid_0's multi_error: 0.156304
[657]	valid_0's multi_error: 0.15544
[658]	valid_0's multi_error: 0.154577
[659]	valid_0's multi_error: 0.154577
[660]	valid_0's multi_error: 0.15544
[661]	valid_0's multi_error: 0.154577
[662]	valid_0's multi_error: 0.154577
[663]	valid_0's multi_error: 0.154577
[664]	valid_0's multi_error: 0.154577
[665]	valid_0's multi_error: 0.154577
[666]	valid_0's multi_error: 0.154577
[667]	valid_0's multi_error: 0.154577
[668]	valid_0's multi_error: 0.154577
[669]	valid_0's multi_error: 0.154577
[670]	valid_0's multi_error: 0.154577
[671]	valid_0's multi_error: 0.154577
[672]	valid_0's

array([[0.00481614, 0.25508974, 0.51616778, ..., 0.00394619, 0.0039539 ,
        0.00392881],
       [0.02456657, 0.22330469, 0.14629457, ..., 0.01057045, 0.01059113,
        0.01058359],
       [0.00369082, 0.31197499, 0.4604887 , ..., 0.00370124, 0.00370847,
        0.00368494],
       ...,
       [0.02028338, 0.00284251, 0.00266086, ..., 0.00267912, 0.00268436,
        0.00268244],
       [0.00509602, 0.00470843, 0.00469748, ..., 0.00472971, 0.00473896,
        0.00473559],
       [0.00271121, 0.0027925 , 0.00270034, ..., 0.00271886, 0.00272418,
        0.00272224]])

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.grid_search import GridSearchCV  # Perforing grid search
from sklearn.model_selection import train_test_split

train_data = pd.read_csv('train.csv')   # 读取数据
y = train_data.pop('30').values   # 用pop方式将训练数据中的标签值y取出来，作为训练目标，这里的‘30’是标签的列名
col = train_data.columns   
x = train_data[col].values  # 剩下的列作为训练数据
train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.333, random_state=0)   # 分训练集和验证集
train = lgb.Dataset(train_x, train_y)
valid = lgb.Dataset(valid_x, valid_y, reference=train)


parameters = {
              'max_depth': [15, 20, 25, 30, 35],
              'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.15],
              'feature_fraction': [0.6, 0.7, 0.8, 0.9, 0.95],
              'bagging_fraction': [0.6, 0.7, 0.8, 0.9, 0.95],
              'bagging_freq': [2, 4, 5, 6, 8],
              'lambda_l1': [0, 0.1, 0.4, 0.5, 0.6],
              'lambda_l2': [0, 10, 15, 35, 40],
              'cat_smooth': [1, 10, 15, 20, 35]
}
gbm = lgb.LGBMClassifier(boosting_type='gbdt',
                         objective = 'binary',
                         metric = 'auc',
                         verbose = 0,
                         learning_rate = 0.01,
                         num_leaves = 35,
                         feature_fraction=0.8,
                         bagging_fraction= 0.9,
                         bagging_freq= 8,
                         lambda_l1= 0.6,
                         lambda_l2= 0)
# 有了gridsearch我们便不需要fit函数
gsearch = GridSearchCV(gbm, param_grid=parameters, scoring='accuracy', cv=3)
gsearch.fit(train_x, train_y)

print("Best score: %0.3f" % gsearch.best_score_)
print("Best parameters set:")
best_parameters = gsearch.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))


In [23]:
a=np.array([[1,2,3],[4,5,6],[7,8,9],[10,11,12]])

In [28]:
a[[1,2,3],:]

array([[ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])